<a href="https://colab.research.google.com/github/sazio/Transients/blob/master/Python/WLC_SSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
from tqdm.notebook import tqdm 

import warnings
warnings.filterwarnings('ignore')

In [229]:
# Simulation's Parameters 
N_s = 576 # x_i with i in [1,...,N_s]
N_p = 8 # a_i with i in [1,...,N_p]
alpha = 1
beta = 2.5
V_1 = 0.9
eps = 0.01
sigma = 0.0001
tau = 5 # delay in competition matrix
V_0 = 5 

In [232]:
t = np.linspace(0, 1, num = 500) # time steps 
# adding some noise to principal neurons initialization
a = np.asarray([np.ones(N_p) + np.random.normal(0,0.0001, N_p) for i in range(0,len(t))]) # principal neurons 

In [233]:
# Generating binary patterns 
np.random.seed(137)
num_patterns = 2
patterns = [np.random.randint(0,2,N_s) for i in range(num_patterns)]

In [234]:
pattern_1 = [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,
             0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0]

patterns.append(np.asarray(pattern_1))

In [235]:
time_each_pattern = len(t)//(num_patterns + 1)

In [236]:
x_0 = np.asarray([patterns[0] for i in range(0,time_each_pattern)]) # sensory neurons -- pattern 1 
x_1 = np.asarray([patterns[1] for i in range(time_each_pattern, 2*time_each_pattern)]) # sensory neurons -- pattern 2
x_2 = np.asarray([patterns[2] for i in range(2*time_each_pattern, len(t))]) # sensory neurons -- pattern 3

x = np.vstack((x_0,x_1,x_2)) # sensory neurons -- all the patterns

In [237]:
np.random.seed(137) # setting random seed  
eta = np.random.normal(0,0.0001, (N_s,N_p)) # small perturbations for initial state
P = np.asarray([np.ones((N_s,N_p)) + eta  for i in range(0,len(t))]) # get shape
#P = np.moveaxis(P, 0, -1) # shape  == (588, 10, 1000)

V = np.asarray([ V_0*np.ones((N_p, N_p)) - (V_0 - 1)*np.eye(N_p) for i in range(0, len(t))]) # competition matrix 
#V = np.moveaxis(V, 0, -1)

xi_t = np.random.uniform(0, sigma, size=len(t)) # adding noise uniformly sampled [0,sigma]

In [238]:
def ODE_Int(a,x,t,P,V,xi_t,alpha = 1):
  """
  To be implemented 
  beta = 2.5
  V_1 = 0.9
  eps = 0.01
  """

  for n in tqdm(range(0,len(t)-1)):
    for i in range(0, N_p):
      a[n+1,i] = (a[n,i] -a[n,i]*np.matmul(V[n, i,:],a[n,:]) + alpha*a[n,i]*np.matmul(P[n,:,:].T[i,:], x[n,:]) + xi_t[n])*(t[n+1] - t[n]) 
      for j in range(0, N_s):
        for k in range(0, N_p):
          P[n+1, j, i] = P[n, j, i] + eps*a[n, i]*(beta*x[n, j] -P[n,j,i])*(t[n+1] - t[n])
          if n-tau >= 0: 
            V[n+1,i, k] = V[n, i, k] + eps*a[n, i]*a[n-tau,k]*(V_1 - V[n, i, k])*(t[n+1] - t[n])
          else:
            V[n+1, i, k] = V[n, i, k] + eps*a[n, i]*a[n, k]*(V_1 - V[n, i, k])*(t[n+1] - t[n])

  return (a, P, V)

In [239]:
#SSM = np.vectorize(ODE_Int,otypes=[np.float],cache=False)

In [ ]:
a_learn, P_learn, V_learn = ODE_Int(a,x,t,P,V,xi_t,alpha = 0)

In [ ]:
a_retr, P_retr, V_retr = ODE_Int(a_learn, x, t, P_learn, V_learn, xi_t, alpha = 1)

In [ ]:
a_mix, P_mix, V_mix = ODE_Int(a,x,t,P,V,xi_t,alpha = 0.5)